In [2]:
#imports
from imgaug import augmenters as iaa
import imgaug as ia
import os
import numpy as np
import cv2 
import glob
from PIL import Image
from imgaug.augmentables.bbs import BoundingBox, BoundingBoxesOnImage


In [3]:
# seed for ia randomizer
ia.seed(1)

In [4]:
# fetch certain file types
images_names = []
ignored = 0

for fname in os.listdir('images'):
    if fname.endswith('.jpg'):
        images_names.append(fname)
    else:
        ignored += 1

print("found", len(images_names), ".jpg files")
print("ignored", ignored, "other file types")

found 9 .jpg files
ignored 9 other file types


In [5]:
# extract only bounding box information from .txt file (see format below)
def extract_bounding_boxes(file):
    bounding_boxes = []
    with open(file) as f:
        boxes = f.readlines()
        for box in boxes:
            box_arr = box.split(' ')            
            bounding_boxes.append((box_arr[1], box_arr[2], box_arr[3], box_arr[4].strip("\n")))
    return bounding_boxes


Bounding box .txt files should look as follows:

3 0.297656 0.615278 0.118750 0.205556   
3 0.622656 0.374306 0.056250 0.093056

Where the first number represents a classification label

In [6]:
# put images in np-array from memory & put corresponding bounding boxes in array in order
# bounding_box_images contains tuples in which the first element corresponds to the image and the second element corresponds to the bounding boxes details as an array
bounding_box_images = []

for image_name in images_names:
    # obtain image
    location = "images/"+image_name
    image = cv2.imread (location)
    
    # obtain bounding boxes
    box_location = os.path.splitext(location)[0] + '.txt'
    bounding_boxes = extract_bounding_boxes(box_location)
    
    # put data in tuple and append to array
    bounding_box_images.append((image, bounding_boxes))



print("images found with bounding box(es):", len(bounding_box_images))



images found with bounding box(es): 9


In [7]:
# sequential operations

seq = iaa.Sequential([
    iaa.Multiply((1.2, 1.5)), # change brightness, doesn't affect BBs
    iaa.Affine(
        translate_px={"x": 40, "y": 60},
        scale=(0.5, 0.7)
    ) # translate by 40/60px on x/y axis, and scale to 50-70%, affects BBs
])

In [8]:
# Augment BBs and images.

# function takes image with corresponding bounding boxes and performs augmentation on both.
def augment_bb_image(img, bbs):
    x, y = seq(image=img, bounding_boxes=bbs)
    return x, y

# transforms pixel data from bounding boxes to normalized yolo values
def normalize_bounding_boxes(img_shape, bbs):
    width = img_shape[0]
    heigth = img_shape[1]
    normalized_bbs = []
    for bb in bbs:   
        bb_normalized = (round(bb[0]/width, 6), round(bb[1]/heigth, 6), round(bb[2]/width, 6), round(bb[3]/heigth, 6))
        normalized_bbs.append(bb_normalized)
    return normalized_bbs

In [9]:
# perform image & bounding box augmentation on all images
augmented_bb_images = []
for bb_img in bounding_box_images:
    # print(len(bb_img[1]))
    # extract bounding boxes per image and create BoundingBox objects from data
    bbs = BoundingBoxesOnImage([
        BoundingBox(x1=bb[0], y1=bb[1], x2=bb[2], y2=bb[3]) for bb in bb_img[1]
    ], shape=bb_img[0].shape)

    # augmented_bb_images
    for bb in bbs:
        print(bb[1])
    aug_img, aug_bbs = augment_bb_image(bb_img[0], bbs)
    
    # normalize bounding boxes to yolo format
    aug_bbs = [(bb.x1, bb.y1, bb.x2, bb.y2) for bb in aug_bbs]
    aug_bbs = normalize_bounding_boxes(bb_img[0].shape, aug_bbs)
    aug_bb_img = (aug_img, aug_bbs)

    augmented_bb_images.append(aug_bb_img)







[0.297656 0.615278]
[0.622656 0.374306]
[0.842969 0.663194]
[0.651563 0.194444]
[0.503906 0.998611]
[0.169922 0.770139]
[0.578125 0.916667]
[0.474219 0.494444]
[0.789453 0.902083]
[0.346484 0.251389]
[0.277344 0.478472]
[0.430859 0.654861]
[0.732812 0.84375 ]
[0.186719 0.715278]
[0.216406 0.394444]
[0.192969 0.1875  ]


In [13]:
# ONLY EXECUTE THIS CELL BLOCK IF YOU WANT THE CURRENT AUGMENTED IMAGES FOLDER TO BE DELETED!!!

# empty augmented_images folder
path = 'augmented_bb_images'
for f in os.listdir(path):
    os.remove(os.path.join(path, f))

In [14]:
# save augmented images & corresponding bounding boxes in the 'augmented_bb_images' folder

for index,img in enumerate(augmented_bb_images):
    
    try:
        # save augmented image
        name = "AUGMENTED_IMG-" + str(index) + ".jpg"
        cv2.imwrite(os.path.join(path , name), img[0])
        
        # save augmented bounding box
        name = path+"\AUGMENTED_IMG-" + str(index) + ".txt"
        print("bbs:", img[1])
        np.savetxt(name, img[1])
    except Exception as ex:
        print(ex)



bbs: [(0.359405, 0.143086, 0.359568, 0.143297), (0.359348, 0.143028, 0.359865, 0.143173)]
bbs: [(0.483326, 0.182265, 0.483875, 0.182475), (0.483436, 0.182246, 0.483737, 0.182285)]
bbs: [(0.466421, 0.177026, 0.466609, 0.177235)]
bbs: [(0.437172, 0.167639, 0.437269, 0.167952)]
bbs: [(0.495259, 0.186124, 0.495388, 0.186275)]
bbs: [(0.435903, 0.167242, 0.436213, 0.167419), (0.436029, 0.167286, 0.436464, 0.167601)]
bbs: [(0.330478, 0.133869, 0.330614, 0.133936), (0.330504, 0.134038, 0.330548, 0.134058), (0.330427, 0.133922, 0.330695, 0.134154), (0.330637, 0.133969, 0.330985, 0.134256), (0.330372, 0.133988, 0.330461, 0.134186)]
bbs: [(0.43365, 0.166568, 0.433736, 0.166656)]
bbs: [(0.402299, 0.156592, 0.402337, 0.156636)]
